In [1]:
import sys
sys.path.append("..")

In [2]:
import pandas as pd
import random
from src.simulation import simulate

In [3]:
assortment_df = pd.read_csv("data/assortment.csv",
                            dtype={"product_id": str,
                                   "product_group": str,
                                   })

orders_df = pd.read_csv("data/orders.csv",
                        parse_dates=['datetime'])

### Assortment
Each row in 'assortment_df' contains a product that we have in our assortment that could be ordered.

Column definitions:
* product_id: product identifier
* product_group: lower-level category of product
* size_group: size of the product
* cluster: higher-level category of product

### Orders
Each row in 'orders_df' contains an shop order line.

Column definitions:
* datetime: datetime when the shop order is placed
* order_id: order identifier
* product_id: product that is ordered

In [4]:
def generate_naive_stock_allocation(assortment):
    warehouse_allocations = random.choices([[True, True],[True, False], [False, True]],
                              k=len(assortment), weights=[0.1, 0.45, 0.45])

    return pd.concat([assortment['product_id'], pd.DataFrame(warehouse_allocations, columns=[
        'warehouseA', 'warehouseB'])], axis=1)

In [5]:
stock_allocation = generate_naive_stock_allocation(assortment_df)
stock_allocation.head()

,product_id,warehouseA,warehouseB
0,007e97352fdb433,False,True
1,32b8b71245fc430,False,True
2,d6825ec8a7d5409,False,True
3,93210367755d441,False,True
4,1302c851692140a,True,False


In [6]:
simulation_result = simulate(orders_df, stock_allocation)
simulation_result.show_aggregated_results()

Warehouse orders sourced: 122005
Total costs: 901680.4
Lost sales: 9103 products


In [7]:
simulation_result.warehouse_orders

,shop_order_id,warehouse_id,pack_line,products,warehouse_costs,shipment_costs,order_datetime,total_costs,quantity
0,00330b8e0df6,warehouseB,monoManual,[83ee1b0d14e148a],0.5,6.0,2021-01-01 18:50:36.719824,6.5,1
1,0349de8a0eb0,warehouseB,multiManual,"[4f14c89b683a428, 9e571016c83f48c, 650c3ad888a...",3.5,6.0,2021-01-01 18:57:30.679387,9.5,5
2,0349de8a0eb0,warehouseA,multiManual,"[286ea9ad60314b5, 589a31255ac5410]",1.8,5.0,2021-01-01 18:57:30.679387,6.8,2
3,051f784d56df,warehouseB,monoManual,[e758471031f34c2],0.5,6.0,2021-01-01 16:59:23.342815,6.5,1
4,06074fe2159d,warehouseA,monoManual,[5d4331c419a24ca],0.7,5.0,2021-01-01 22:35:34.041543,5.7,1
...,...,...,...,...,...,...,...,...,...
122000,fa0858a42cba,warehouseB,monoManual,[810c38c9a7e94fe],0.5,6.0,2021-12-30 20:54:44.372585,6.5,1
122001,fd3d162c659a,warehouseB,monoManual,[64a912c7dff64bf],0.5,6.0,2021-12-30 12:23:37.006524,6.5,1
122002,fda72877f233,warehouseB,monoManual,[0b64c0ea4f68450],0.5,6.0,2021-12-30 11:00:19.365373,6.5,1
122003,fe5c14b218f3,warehouseB,monoManual,[3ef8dc897dfc4d2],0.5,6.0,2021-12-30 15:45:00.543150,6.5,1


In [8]:
simulation_result.lost_sales

,shop_order_id,order_datetime,product_id
0,f9ed0f74dde7,2021-01-01 13:04:20.896700,1dd8ba3c4b6f429
1,ff60173b69f5,2021-01-01 10:09:50.076472,7fbc37010e1249e
2,e9f22a50e42d,2021-01-05 18:24:38.694948,8428d086f8c8435
3,eb3031b98a19,2021-01-05 16:28:14.456537,4ffe48858f76491
4,ebaf66c3a346,2021-01-05 21:48:59.205404,3ebbbcfdb6f346f
...,...,...,...
9098,f7c320471c57,2021-12-30 19:41:55.488927,545154f87c24418
9099,f889ca44c6b5,2021-12-30 15:53:16.923278,9b4548b4ff1b46c
9100,fc4a4217c8d0,2021-12-30 14:22:56.930373,b7236e8db94e4a8
9101,fefdccbd189c,2021-12-30 20:45:04.570772,86cf47783e0342b


# Base model : Distrubute According to Cluster

In [9]:

#Initialize an empty list to store rows
warehouse_rows = []

# Iterate over rows in product_df and populate the warehouse_df based on conditions
for index, row in assortment_df.iterrows():
    product_id = row['product_id']
    cluster = row['cluster']

    # Initialize warehouseA and warehouseB as False
    warehouseA = False
    warehouseB = False

    # Check conditions and update warehouse columns
    if cluster in ['toy', 'garden', 'winter']:
        warehouseA = True
    elif cluster in ['clothes']:
        warehouseB = True

    # Append a new row to the list
    warehouse_rows.append({'product_id': product_id, 'warehouseA': warehouseA, 'warehouseB': warehouseB})

# Create a DataFrame from the list of rows
warehouse_df = pd.DataFrame(warehouse_rows)

# Display the resulting warehouse_df
warehouse_df

,product_id,warehouseA,warehouseB
0,007e97352fdb433,False,False
1,32b8b71245fc430,False,False
2,d6825ec8a7d5409,False,False
3,93210367755d441,False,False
4,1302c851692140a,False,False
...,...,...,...
995,fd27949fe39645c,True,False
996,52047e1f1e96465,True,False
997,8028747200e8443,True,False
998,e1bac8e0a574429,True,False


In [11]:
simulation_result = simulate(orders_df, warehouse_df)
simulation_result.show_aggregated_results()

Warehouse orders sourced: 92311
Total costs: 1270093.1
Lost sales: 67608 products


# Distrubute According to Total Sales of the Items

In [12]:
# Merge the DataFrames on the 'product_id' column
merged_df = pd.merge(orders_df, assortment_df, on='product_id', how='left')

# Calculate total sales for each product_id
total_sales_df = merged_df['product_id'].value_counts().reset_index()
total_sales_df.columns = ['product_id', 'total_sales']

# Sort the DataFrame by total sales in descending order
total_sales_df = total_sales_df.sort_values(by='total_sales', ascending=False)

# Display the resulting DataFrame with total sales sorted
total_sales_df

,product_id,total_sales
0,03b755d783fd4ec,644
1,62e3040b6322463,639
2,6b91277c742e439,634
3,36103fff52e640a,630
4,93210367755d441,627
...,...,...
995,0f44a9e6585b450,72
996,55a3f2db0b8846a,71
997,f98dbfdfecb7464,67
998,6e4120fb6f16405,65


In [13]:
# Find percentiles of the total sales
percentiles_df = total_sales_df['total_sales'].quantile([0.2,0.4, 0.5,0.6, 0.75,0.8,0.90, 1]).reset_index(name='percentile')
percentiles_df

,index,percentile
0,0.20,97.0
1,0.40,142.4
2,0.50,193.0
3,0.60,202.0
4,0.75,217.0
5,0.80,222.0
6,0.90,281.5
7,1.00,644.0


In [14]:

#Initialize an empty list to store rows
warehouse_rows = []

# Iterate over rows in product_df and populate the warehouse_df based on conditions
for index, row in total_sales_df.iterrows():
    product_id = row['product_id']
    total_sales = row['total_sales']

    # Initialize warehouseA and warehouseB as False
    warehouseA = False
    warehouseB = False

    # Check conditions and update warehouse columns
    if (total_sales < 193) or (total_sales > 282):
        warehouseB = True
    if total_sales >= 142:
        warehouseA = True

    # Append a new row to the list
    warehouse_rows.append({'product_id': product_id, 'warehouseA': warehouseA, 'warehouseB': warehouseB, 'total_sales':total_sales})

# Create a DataFrame from the list of rows
warehouse_df = pd.DataFrame(warehouse_rows)

# Display the resulting warehouse_df
warehouse_df

,product_id,warehouseA,warehouseB,total_sales
0,03b755d783fd4ec,True,True,644
1,62e3040b6322463,True,True,639
2,6b91277c742e439,True,True,634
3,36103fff52e640a,True,True,630
4,93210367755d441,True,True,627
...,...,...,...,...
995,0f44a9e6585b450,False,True,72
996,55a3f2db0b8846a,False,True,71
997,f98dbfdfecb7464,False,True,67
998,6e4120fb6f16405,False,True,65


In [15]:
simulation_result = simulate(orders_df, warehouse_df)
simulation_result.show_aggregated_results()

Warehouse orders sourced: 104706
Total costs: 885333.5
Lost sales: 18222 products


In [19]:
100*(901680-885333)/901680

1.8129491615650786

### Improvement: 1.81%

# Distrubute According to Products that Sold Togetter: Embeddings

In [16]:
most_sold_items_df = pd.read_csv("most_sold_items.csv")

remaining_items_df = pd.read_csv("remaining_items.csv")

In [17]:
#Initialize an empty list to store rows
warehouse_rows = []
remaining_items = remaining_items_df["product_id"].tolist()
most_sold_items = most_sold_items_df["product_id"].tolist()
# Iterate over rows in product_df and populate the warehouse_df based on conditions
for index, row in assortment_df.iterrows():
    product_id = row['product_id']
   
    # Initialize warehouseA and warehouseB as False
    warehouseA = False
    warehouseB = False

    # Check conditions and update warehouse columns
    if product_id in most_sold_items:
        warehouseB = True
    if product_id in remaining_items:
        warehouseA = True

    # Append a new row to the list
    warehouse_rows.append({'product_id': product_id, 'warehouseA': warehouseA, 'warehouseB': warehouseB})

# Create a DataFrame from the list of rows
warehouse_df = pd.DataFrame(warehouse_rows)

# Display the resulting warehouse_df
warehouse_df

,product_id,warehouseA,warehouseB
0,007e97352fdb433,True,True
1,32b8b71245fc430,True,True
2,d6825ec8a7d5409,True,True
3,93210367755d441,True,True
4,1302c851692140a,True,True
...,...,...,...
995,fd27949fe39645c,False,True
996,52047e1f1e96465,False,True
997,8028747200e8443,True,False
998,e1bac8e0a574429,False,True


In [18]:
simulation_result = simulate(orders_df, warehouse_df)
simulation_result.show_aggregated_results()

Warehouse orders sourced: 120221
Total costs: 851057.1
Lost sales: 4672 products


### Improvement: 5.61%

In [20]:
100*(901680-851057)/901680

5.614297755301216